# OpenCV Camera

In [1]:
# reload library
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
np.set_printoptions(precision=1)
np.set_printoptions(suppress=True)

import cv2

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10) # set default figure width

from slurm import storage
from pprint import pprint
from pathlib import Path

In [4]:
import opencv_camera as oc

In [5]:
!ls

3d-pics   floor    imgs3	  stereo-camera-2.ipynb  table4
cal_pics  grab.py  misc		  stereopics		 test.py
cb.ply	  imgs1    pics		  table			 videogames
docs	  imgs2    stereo2.ipynb  table2


In [45]:
# See the answer from OpenCV Tutorials
ans = cv2.FileStorage("cal_pics/intrinsics.yml", cv2.FILE_STORAGE_READ)
print("Left camera K1 and D1")
print("-"*40)
print(ans.getNode("M1").mat())
print(ans.getNode("D1").mat())
print("")
print("Right camera K2 and D2")
print("-"*40)
print(ans.getNode("M2").mat())
print(ans.getNode("D2").mat())

Left camera K1 and D1
----------------------------------------
[[534.8   0.  335.7]
 [  0.  534.8 240.7]
 [  0.    0.    1. ]]
[[ 0.3 -1.   0.   0.   0. ]]

Right camera K2 and D2
----------------------------------------
[[534.8   0.  334.6]
 [  0.  534.8 242.1]
 [  0.    0.    1. ]]
[[-0.2 -0.1  0.   0.   0. ]]


In [13]:
def readImages(pattern):
    p = Path("cal_pics").glob(pattern)
    return [cv2.imread(str(f),0) for f in p]

imgL = readImages("left*.jpg")
imgR = readImages("right*.jpg")

In [48]:
# plt.imshow(imgL[0], cmap="gray")
# plt.title(f"{imgL[0].shape}");

In [18]:
from opencv_camera import CameraCalibration
from opencv_camera.stereo_calibrate import StereoCalibration
from opencv_camera.targets.chessboard import ChessboardFinder
from opencv_camera import UnDistort
from opencv_camera.color_space import bgr2gray, gray2bgr

In [107]:
stereoCal = StereoCalibration()
board = ChessboardFinder((9,6), 1)

In [108]:
ok, cm, sc = stereoCal.calibrate(imgL, imgR, board)

Images: 13 @ (480, 640)
Chessboard (9, 6)
----------------------------------------
[0] + found 54.0 of 54 corners
[1] + found 54.0 of 54 corners
[2] + found 54.0 of 54 corners
[3] + found 54.0 of 54 corners
[4] + found 54.0 of 54 corners
[5] + found 54.0 of 54 corners
[6] + found 54.0 of 54 corners
[7] + found 54.0 of 54 corners
[8] + found 54.0 of 54 corners
[9] + found 54.0 of 54 corners
[10] + found 54.0 of 54 corners
[11] + found 54.0 of 54 corners
[12] + found 54.0 of 54 corners

RMS error: 0.1954335878493804

----------------------------------------
Images: 13 @ (480, 640)
Chessboard (9, 6)
----------------------------------------
[0] + found 54.0 of 54 corners
[1] + found 54.0 of 54 corners
[2] + found 54.0 of 54 corners
[3] + found 54.0 of 54 corners
[4] + found 54.0 of 54 corners
[5] + found 54.0 of 54 corners
[6] + found 54.0 of 54 corners
[7] + found 54.0 of 54 corners
[8] + found 54.0 of 54 corners
[9] + found 54.0 of 54 corners
[10] + found 54.0 of 54 corners
[11] + found 

In [110]:
print(sc)

Camera 1 --------------------------
  focalLength(x,y): 564.0 571.4 px 
  principlePoint(x,y): 332.7 313.1 px
  distortionCoeffs: [ -1.598   5.686  -0.174   0.108 -17.621]
Camera 2 --------------------------
  focalLength(x,y): 564.0 571.4 px 
  principlePoint(x,y): 350.9 113.0 px
  distortionCoeffs: [-0.466  7.828  0.231  0.484 -8.676]
Extrinsic Camera Parameters -------
  Translation between Left/Right Camera: [-9.641 13.95  10.019]
  Rotation between Left/Right Camera:
    [[ 0.947 -0.139  0.289]
     [ 0.321  0.391 -0.863]
     [ 0.006  0.91   0.415]]
  Essential Matrix:
    [[ -3.126   8.776  14.428]
     [  9.55    7.384   6.898]
     [-16.305  -1.837   4.279]]
  Fundatmental Matrix:
    [[-0.     0.     0.002]
     [ 0.     0.    -0.001]
     [-0.003 -0.002  1.   ]]



In [129]:
from numpy.linalg import inv, norm

def computeF(K1, K2, R, t):
    """
    OpenCV appears to normalize the F matrix. It can contain
    some really large and really small values. See CSE486
    lecture 19 notes [1] for examples of the F matrix.
    Also see Epipolar Geometery, Table 9.1, last line (Cameras 
    not at inf)[2] for how to calculate it.
    
    [1] http://www.cse.psu.edu/~rtc12/CSE486/lecture19.pdf
    [2] http://www.robots.ox.ac.uk/~vgg/hzbook/hzbook2/HZepipolar.pdf
    """
    t = t.T[0]
    A = (K1 @ R.T @ t)
    print(A)
    C = np.array([
        [    0, -A[2], A[1]],
        [ A[2],     0,-A[0]],
        [-A[1],  A[0],    0]
    ])
    ret = inv(K2).T @ R @ K1.T @ C
    return ret/norm(ret)

In [132]:
ee = computeF(sc.K1, sc.K2, sc.R, sc.T)
print(ee)

[-6139.9    5747.431   -10.669]
[[-0.     0.     0.002]
 [ 0.     0.    -0.001]
 [-0.003 -0.002  1.   ]]


In [134]:
ee @ np.array([30,20,1])

array([ 0.002, -0.   ,  0.859])